In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def make_late(N, beta):
    """Creates an NxN array with elements of value beta with an ice seed (a 1) 
    planted at the centre of the grid
        
    Input:
    
    * N: Odd nteger 
    
    * beta: Float >=0
    
    Output:
    
    * NxN array with element "1" planted at pos
    """
    
    # Creates array with beta values
    array = np.full((N, N), beta)
    
    pos = int((N - 1)/2) # Defines centre of grid for seed to be planted
    
    # If N isn't odd
    if N % 2 == 0:
        
        print("N must be an odd integer greater than 1")
    
    # If N is odd
    else:
        
        # Plants one ice seed (a 1) at the centre
        array[pos][pos] = 1
    
    return array

def make_hex_grid(N):
    """Creates coordinate matrices for a hexagonal grid by shifting each even row
    to the right by 0.5 and compresses the y-axis down to 
    sqrt(3) / 2 of its scale
    
    Inputs:
    
    * N: Integer, extent of coordinate vectors
    
    """
    
    x = np.arange(0, float(N)) 
    
    y = x
    
    X, Y = np.meshgrid(x, y)
    
    Y = Y * np.sqrt(3) / 2
    
    X[::2, :] += 0.5
    
    return X, Y

In [ ]:
def update_hex(lattice, alpha, beta, gamma, max_time):
    """Creates a snowflake on a hexagonal lattice unitl a maximum
    timestep max_time or until ice reaches the boundary along the centre
    axis
    
    Inputs:
    
    * lattice: Array
    
    * alpha, beta, gamma: Floats
    
    * max_time: Integer
    
    Outputs:
    
    * lat: Array, updated lattice
    """
    
    # Initialise timestep
    t = 0

    lat = np.copy(lattice)
    
    N = np.shape(lat)[0]
    
    # Takes the max value of all the values just before the boundary
    boundary = np.max(np.concatenate((np.ravel(lat[1, 1:-2]), np.ravel(lat[-2, 1:-2]), np.ravel(lat[1:-2, 1]), np.ravel(lat[1:-2, -2]))))
    
    # Will run until max_time or ice reaches boundary
    while t <= max_time and boundary < 1  :
        
        t += 1

        # Identifies ice 
        ice = np.where(lat >= 1, lat, 0)
        
        # Left neighbour
        left = np.where(np.roll(ice, -1) >= 1, lat, 0) 
        
        # Right neighbour
        right = np.where(np.roll(ice, 1) >= 1, lat, 0) 
        
        # Up. neighbour
        top = np.where(np.roll(ice, -1, axis = 0) >= 1, lat, 0) 
        
        # Bottom neighbour
        bottom = np.where(np.roll(ice, 1, axis = 0) >= 1, lat, 0) 
        
        # U.diag neighbours on odd rows
        odd_ud = np.where(np.roll(ice, (1, 1), axis = (0, 1)) >= 1, lat, 0) 
        
        # B. diag. neighbours on odd rows
        odd_bd = np.where(np.roll(ice, (-1, 1), axis = (0, 1)) >= 1, lat, 0) 
        
        # Sets every element on odd rows to zero 
        odd_ud[0::2, :] = odd_bd[0::2, :] = 0
        
        # U.diag neighbour on even rows
        even_ud = np.where(np.roll(ice, (1, -1), axis = (0, 1)) >= 1, lat, 0) 
        
        # B. diag. neighbours on even rows
        even_bd = np.where(np.roll(ice, (-1, -1), axis = (0, 1)) >= 1, lat, 0) 
        
        # Sets every element on the even rows to zero
        even_ud[1::2, :] = even_bd[1::2, :] = 0
        
        # Adds the neighbours of the identified ice sites to get the receptive sites
        r1 = np.where(ice == 0, ice + left, ice)
        
        r2 = np.where(r1 == 0, r1 + right, r1)
        
        r3 = np.where(r2 == 0, r2 + top, r2)
        
        r4 = np.where(r3 == 0, r3 + bottom, r3)
        
        r5 = np.where(r4 == 0, r4 + even_bd, r4)
        
        r6 = np.where(r5 == 0, r5 + even_ud, r5)
        
        r7 = np.where(r6 == 0, r6 + odd_ud, r6)
        
        r8 = np.where(r7 == 0, r7 + odd_bd, r7)
        
        receptive_sites =  r8
        
        # Adds constant to receptive sites
        receptive_sites = np.where(receptive_sites != 0, receptive_sites + gamma, 0)
        
         # Reinforces the boundary conditions
        receptive_sites[:, 0] = receptive_sites[:, -1] = receptive_sites[0, :] = receptive_sites[-1, :] = beta
        
        # Identifies unreceptive sites and returns lattice with receptive sites set to zero
        unreceptive_sites = np.where(receptive_sites == 0, lat, 0)
        
        # Reinforces the boundary conditions
        unreceptive_sites[:, 0] = unreceptive_sites[:, -1] = unreceptive_sites[0, :] = unreceptive_sites[-1, :] = beta
        
        # Calculates the sum of the neighbours on odd rows
        son_odd = (
            
            np.roll(unreceptive_sites, 1) + # Left neighbour
           np.roll(unreceptive_sites, -1) + # Right neighbour 
           np.roll(unreceptive_sites, 1, axis = 0) + # Top neighbour
           np.roll(unreceptive_sites, -1, axis = 0) + # Bottom neighbour
           np.roll(unreceptive_sites, (1, 1), axis = (0, 1)) + # U. diagonal neighbour
           np.roll(unreceptive_sites, (-1, 1), axis = (0, 1)) # B. diagonal neighbour
            )
        
        # Sets all neighbour evaluations on even rows to zero
        son_odd[::2, :] = 0
        
        # Calculates the sum of the neighbours on even rows
        son_even = (
            
            np.roll(unreceptive_sites, 1) + # Left neighbour
           np.roll(unreceptive_sites, -1) + # Right neighbour 
           np.roll(unreceptive_sites, 1, axis = 0) + # Top neighbour
           np.roll(unreceptive_sites, -1, axis = 0) + # Bottom neighbour
           np.roll(unreceptive_sites, (1, -1), axis = (0, 1)) + # U. diagonal neighbour
           np.roll(unreceptive_sites, (-1, -1), axis = (0, 1)) # B. diagonal neighbour
            )
        
        # Sets all neighbour evaluations on odd rows to zero
        son_even[1::2, :] = 0
        
        # Lattice containin the sum of the neighbours at each lattice point
        son = son_even + son_odd
        
        # Reinforces the boundary conditions
        son[:, 0] = son[:, -1] = son[0, :] = son[-1, :] = beta
        
        # Applies diffusion equation to the unreceptive sites
        unreceptive_sites += alpha * (-6 * unreceptive_sites + son) / 12
        
        # Reinforces the boundary conditions
        unreceptive_sites[:, 0] = unreceptive_sites[:, -1] = unreceptive_sites[0, :] = unreceptive_sites[-1, :] = beta

        # Adds updated receptive and unreceptive sites
        fin_lat = receptive_sites + unreceptive_sites
        
        # Reinforces the boundary conditions
        fin_lat[:, 0] = fin_lat[:, -1] = fin_lat[0, :] = fin_lat[-1, :] = beta
        
        lat = fin_lat
        
        boundary = np.max(np.concatenate((np.ravel(lat[1, 1:-2]), np.ravel(lat[-2, 1:-2]), np.ravel(lat[1:-2, 1]), np.ravel(lat[1:-2, -2]))))
        
    # Returns the final lattice 
    return lat, t

In [ ]:
# E.g.

N = 401
alpha = 2.069
beta = 0.5
gamma = 0.002

X, Y = make_hex_grid(N)

lat = make_late(N, beta)

C, time = update_hex(lat, alpha, beta, gamma, 10000)

In [ ]:
plt.figure(figsize = (16, 14))

plt.scatter(X, Y, s = 5, c = -np.where(C >= 1, C, 0), cmap = "twilight")

plt.title(fr"$\alpha$ = {alpha}, $\beta$ = {beta}, $\gamma$ = {gamma}, timesteps = {time}")